# Model training

## 1.1 Import data and required packages

### Importing pandas, numpy, matplotlib, seaborn and warnings libraries

In [17]:
# basic import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

### Import csv dataset as pandas dataframe

In [9]:
df = pd.read_csv('data/StudentsPerformance.csv')

Show top 5 records

In [10]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


Preparing X and Y variables

In [12]:
X = df.drop(columns=['math score'], axis = 1)
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [13]:
y = df['math score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math score, dtype: int64

In [ ]:
# Create column transformer with 3 types of transformers

num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [("OneHotEncoder", oh_transformer, cat_features),
     ("StandardScaler", numeric_transformer, num_features)
     ]
)

In [15]:
X = preprocessor.fit_transform(X)

In [16]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [18]:
X.shape

(1000, 19)

In [19]:
# separate dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 19), (200, 19), (800,), (200,))

Create a evaluate function to give all metrics after model training

In [20]:
def evaluate_model(true, predicted):
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(true, predicted)
    r2_square = r2_score(true, predicted)
    return mse, rmse, mae, r2_square

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "KNeighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "Support Vector Regressor": SVR(),
    "CatBoost Regressor": CatBoostRegressor(verbose=False),
    "XGB Regressor": XGBRegressor()
}

model_list = []
r2_list = []

for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train) #train model

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    mse_train, rmse_train, mae_train, r2_square_train = evaluate_model(y_train, y_train_pred)
    mse_test, rmse_test, mae_test, r2_square_test = evaluate_model(y_test, y_test_pred)

    model_list.append(list(models.keys())[i])

    print("Model performance for training set")
    print(f"{list(models.keys())[i]}: ")
    print(f"MSE: {mse_train}, RMSE: {rmse_train}, MAE: {mae_train}, R2 Score: {r2_square_train}")
    print("--------------------------------------------------")

    print("Model performance for testing set")
    print(f"{list(models.keys())[i]}: ")
    print(f"MSE: {mse_test}, RMSE: {rmse_test}, MAE: {mae_test}, R2 Score: {r2_square_test}")
    r2_list.append(r2_square_test)
    print("==================================================")

    print("="*35)
    print("\n")

Model performance for training set
Linear Regression: 
MSE: 28.334870380648585, RMSE: 5.323050852720513, MAE: 4.266711846071957, R2 Score: 0.8743172040139593
--------------------------------------------------
Model performance for testing set
Linear Regression: 
MSE: 29.095169866715494, RMSE: 5.393993869732843, MAE: 4.214763142474852, R2 Score: 0.8804332983749564
Model performance for training set
Lasso: 
MSE: 43.47840400585578, RMSE: 6.593815587795565, MAE: 5.206302661246526, R2 Score: 0.8071462015863458
--------------------------------------------------
Model performance for testing set
Lasso: 
MSE: 42.50641683841163, RMSE: 6.519694535667421, MAE: 5.157881810347763, R2 Score: 0.8253197323627852
Model performance for training set
Ridge: 
MSE: 28.337788233082456, RMSE: 5.323324922741656, MAE: 4.264987823725977, R2 Score: 0.8743042615212908
--------------------------------------------------
Model performance for testing set
Ridge: 
MSE: 29.05627219234824, RMSE: 5.390387016935634, MAE: 4